# 1. Introduction: merge and append economic data

Welcome to the SSRIC Instructional Modules for the project, "Teaching Statistics and Economic Data Analysis in Python with Jupyter Notebooks", by Daniel MacDonald, Associate Professor and Chair, Economics Department, CSU San Bernardino. These were written in Summer 2023.

Most of the modules draw extensively on Kevin Sheppard's e-book, *Introduction to Python for Econometrics, Statistics, and Data Analysis*, available here: https://bashtage.github.io/kevinsheppard.com/files/teaching/python/notes/python_introduction_2021.pdf. 

Rather than begin instruction in Python through the core tools of computer programming (such as conditions, loops, and functions), Sheppard begins with Python's major "containers", or data structures. Through practice, I have learned that this is an effective method for teaching Python to economics majors.

The learning objectives of this set of Instructional Modules are as follows. By the end of these modules, students will be able to...

1. Create data structures in Python based on economic data
1. Summarize the statistical properties of economic data (median, mean, max, min, correlation) using Python
1. Create and manage economic data: create new columns and rows, merge and append, and import data from .csv and .xlsx files into Python
1. Visualize economic data using line and scatterplots

The objectives/content of Module 5 are as follows. By the end of this module, students will be able to...

1. Merge and append economic datasets in small-scale cases
1. Perform a larger-scale merge operations on an economic dataset
1. Using `merge` and DataFrame operations to calculate job growth

# 1. Merge and join/append

Merge and join operations combine datasets, either by linking rows or simply adding one variable to another. They are very common in economic data analysis.

We'll start with a very simple example, and then gradually build on the methods to include a larger dataset.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df1=pd.read_excel('module 5 data.xlsx', sheet_name='urate_2022')
df2=pd.read_excel('module 5 data.xlsx', sheet_name='urate_2021')

In [ ]:
print(df1)
print(df2)

## 1.1 Comment on the above

It is very common in business and economic applications to have data in multiple spreadsheets. The code above addresses a very simple example where you might have unemployment rates for a given set of cross sections across two different periods in two separate Excel worksheets. **The goal is to find some way of combining these datasets for analysis**. 

Note that there are two ways that you might want these data combined:

1. You might want it in "long" format, meaning you have a single column with data that refers to the cross section (here, county) and periods of interest
    1. You can think of this as "stacking" the data
1. You might want it in "wide" format, where you have two columns of data, one for each period, where a single row can be used to refer to all information for a particular cross section
    1. Think of this as "joining" the data

We will address each of these ways below.

# 2. Task 1: Append DataFrames (long format)

If you want to append one dataset to another, which is equivalent to the long format, use the `concat` function.

However, to append the dataframes above, there is a twist: we need to create a "date" column that distinguishes one set of observations from another, and we need to rename the columns so that they are the same - that way `concat` can seamlessly join the two dataframes together: 

In [ ]:
df1.rename(columns={'urate_nov2022': 'urate'}, inplace=True)
df1['date']='Nov-2022' #Creates a new column in df1, all values of which equal the string 'Nov-2022'

df2.rename(columns={'urate_nov2021': 'urate'}, inplace=True)
df2['date']='Nov-2021'

print(df1, '\n', df2)

**The most important thing about the "new and improved" df1 and df2 is that each DataFrame has the same column names**. Thus, we should now be able to "stack" the first on the second without any issue.

We want to "stack" rows on rows, which is why in the `pd.concat()` below, we specify `axis=0`. 

If we wanted to "stack" on columns, we'd specify option `axis=1` - but we'll see why, later, that's not usually a good idea.

In [ ]:
df_appended=pd.concat([df1, df2], axis=0) #The solution to the problem - the datasets are now "stacked" accordingly

print(df_appended)
print('Number of observations in new DataFrame:', len(df_appended), '\n')
print(df_appended[df_appended['Area']=='San Luis Obispo'])

# 3. Task 2: Merge DataFrames (create wide format data)

Merging data is different from appending them. In a **merge, you want to join two datasets, usually based on a common variable in both datasets**.

While `concat` can also be used to do a merge-like operation (for example, in the above code, you could adjust the axis in `pd.concat` to axis=1), it's not a true merge because concat does not pay any attention to the other columns of data - it just "stacks" or "smushes" them together - but instead of one on top of each other, it places them next to each other.

Look below at df1 and df2, closely. Notice the rows in df1 are not in the same order as df2. 

I suppose it's possible we could simply sort the rows in df1 and df2, but what if some Counties in df2 are missing from df1? Then a `concat` clearly would not work. We need a better method:

In [ ]:
df1=pd.read_excel('module 5 data.xlsx', sheet_name='urate_2022')
df2=pd.read_excel('module 5 data.xlsx', sheet_name='urate_2021')

print(df1, '\n', df2)

## 3.1 Merge operation: we need a "key"

We need a way to link df1 and df2 without worrying about whether the rows perfectly match up, or whether rows might be missing from one or the other DataFrame.

We need a key that will help us "match" observations from one DataFrame to the other.

The key in this case should be clear - **we need to match on the column `Area`**. Thus, we perform a new function `merge()` on our two DataFrames, and as an option, we specify `on='Area'` to specify the key:

In [ ]:
df_merged=pd.merge(df1, df2, on='Area') #The syntax is intuitive - use 'on' to specify which column is common to both dataframes

print(df_merged)

## 3.2 Merge operations on indexes

As mentioned in previous modules, one of the benefits of the DataFrame structure is the `index` object. 

If you have two dataframes with a common index, you can merge on the index instead of a separate column. See the code below:

In [ ]:
df1=pd.read_excel('module 5 data.xlsx', sheet_name='urate_2022', index_col='Area')
df2=pd.read_excel('module 5 data.xlsx', sheet_name='urate_2021', index_col='Area')

df_merged=pd.merge(df1, df2, left_index=True, right_index=True)

The above example is somewhat contrived, since we saw before that we didn't need to merge on an index in order to solve our problem. But in many economic examples, DataFrames will come with an index, and it's important to see how merge works in these cases.

Once the data are merged, you can calculate changes or percentage changes accordingly:

In [ ]:
df_merged['urate_change']=df_merged['urate_nov2022']-df_merged['urate_nov2021']

print(df_merged)
print(df_merged.loc[['Riverside', 'San Bernardino']])

Note that instead of setting an index column and merging on it, you could also merge in the first way we merged, and then `set_index()` using the matched column to accomplish the same task.

# 4. Try it yourself

Consider the two DataFrames below. Perform the same two operations we did above - but do them in reverse order

1. Merge the dataframes using the 'FIPS' key (call this dataframe `df_merged`)
1. Append the dataframes (call this dataframe `df_appended`)
    1. Please note that to do this, you will need to make sure the column names in both dataframes match (use `.rename()`)

When you are done, `print` both dataframes.

In [ ]:
econ_dict1={'fips':[40140, 12540, 20940, 25260, 31080, 41740, 42020, 37100], 
           'urate_2022': [3.6, 6.7, 14.8, 6.8, 3.9, 2.9, 2.4, 3.2]}

econ_dict2={'fips':[12540, 40140, 25260, 20940, 31080,  37100, 42020, 41740], 
           'urate_2021': [7.4, 5.4, 7.4, 15.1, 5.7, 4.5, 3.5, 4.5]}

df1=pd.DataFrame(econ_dict1)
df2=pd.DataFrame(econ_dict2)

print(df1, '\n\n', df2)

#Perform a merge of the dataframes below:


In [ ]:
econ_dict1={'fips':[40140, 12540, 20940, 25260, 31080, 41740, 42020, 37100], 
           'urate_2022': [3.6, 6.7, 14.8, 6.8, 3.9, 2.9, 2.4, 3.2]}

econ_dict2={'fips':[12540, 40140, 25260, 20940, 31080,  37100, 42020, 41740], 
           'urate_2021': [7.4, 5.4, 7.4, 15.1, 5.7, 4.5, 3.5, 4.5]}

df1=pd.DataFrame(econ_dict1)
df2=pd.DataFrame(econ_dict2)

#Perform an `append` below (don't forget to rename your columns and add a "date" column!):



## 4.1 `merge` final comments

Pandas' `merge` can handle many other cases, where you might be merging "1 to many", or other problems where the data don't precisely line up. I encourage you to explore the Pandas documentation for `merge` and general Google searches or Chat GPT: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html.

# 5. Working with larger datasets

We now learn how to use the methods we just learned to handle larger datasets, such as one for all MSAs in California:

In [ ]:
df1=pd.read_excel('module 5 data.xlsx', sheet_name='Q2_2022')
print(df1.columns)
df1.rename(columns={'june_employment': 'june_employment_2022'}, inplace=True)

df2=pd.read_excel('module 5 data.xlsx', sheet_name='Q2_2021')
print(df2.columns)
df2.rename(columns={'june_employment': 'june_employment_2021'}, inplace=True)

In [ ]:
print(df1, '\n', df2)

These data are from the Bureau of Labor Statistics' Quarterly Census of Employment and Wages. This is an openly accessible dataset that contains detailed county- and industry-level information on employment and wages. The data are used very often in economics research.

**If we want to calculate the year-over-year percentage change in employment for each county, we will need to merge the two years of data first:**

In [ ]:
df_merged=pd.merge(df1, df2, on='area_code')
print(df_merged)

## 5.1 Comment on the above

If you `print` the merged dataframe above, you'll see a lot of columns and information. Since we only want information on the percentage change in employment, we can be more selective 

For example, we obviously don't want all the columns from both datasets - is there any way to just pull the "june employment 2021" information into df1?

In [ ]:
df_merged=pd.merge(df1, df2[['area_code', 'june_employment_2021']], on='area_code')
df_merged['employment_growth']=100*(df_merged['june_employment_2022'] 
                                    - df_merged['june_employment_2021'])/df_merged['june_employment_2021']

In [ ]:
print(df_merged[['area', 'employment_growth']])

## 5.2 Try it yourself: use Q3 2022 and Q3 2021 data

I created two small worksheets that are subsets of the QCEW data above, for California, by metro area. With these data, perform the following tasks:

1. Merge **the 2021 "september employment" data only** onto the 2022 data, using the "area code" key 
1. Calculate the employment growth between the two years
1. Print the data frame - print only the 'area' and 'employment_growth' columns
1. Print the median employment growth
1. Print the maximum employment growth

In [ ]:
ca_msa=[12540, 17020, 20940, 23420, 25260, 31080, 32900, 33700, 33700, 42220, 42280, 44700, 
        40140, 40900, 41860, 41740, 41860, 41940, 42020, 42200, 42220, 42260, 44740, 46700, 47300, 49700]

df1=pd.read_excel('module 5 data.xlsx', sheet_name='Q3_2022')
df1=df1[df1['area_code'].isin(ca_msa)]
df1.rename(columns={'sept_employment': 'sept_employment_2022'}, inplace=True)

df2=pd.read_excel('module 5 data.xlsx', sheet_name='Q3_2021')
df2=df2[df2['area_code'].isin(ca_msa)]
df2.rename(columns={'sept_employment': 'sept_employment_2021'}, inplace=True)

print(df1, df2)

#Try it yourself. Write your code to solve 1-5 below:



# Summary

The `concat` and `merge` functions are very often used in real-world applications. We have only scratched the surface of these applications - especially in the case of `merge`, where many other cases can be addressed. You are encouraged to read the documentation on these functions or search Google or StackExchange (or ask an AI!) to learn more.